# Proyecto SQL: Análisis del Mercado de Libros

## 1. Objetivos del Proyecto

Este proyecto busca analizar datos relacionados con libros, editoriales, autores, calificaciones y reseñas para generar propuestas de valor para un nuevo producto en el mercado de aplicaciones para amantes de los libros. Los objetivos específicos son:

1. **Encontrar el número de libros publicados después del 1 de enero de 2000.**
2. **Determinar el número de reseñas de usuarios y la calificación promedio para cada libro.**
3. **Identificar la editorial que ha publicado el mayor número de libros con más de 50 páginas.**
4. **Identificar al autor con la más alta calificación promedio de libros (considerando solo libros con al menos 50 calificaciones).**
5. **Encontrar el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros.**


### Codigo para conectarse a la base de datos:

In [1]:
import pandas as pd
from sqlalchemy import create_engine

db_config = {
    'user': 'practicum_student', 
    'pwd': 'QnmDH8Sc2TQLvy2G3Vvh7',
    'host': 'yp-trainers-practicum.cluster-czs0gxyx2d8w.us-east-1.rds.amazonaws.com',
    'port': 5432,
    'db': 'data-analyst-final-project-db'
}

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db']
)

engine = create_engine(connection_string, connect_args={'sslmode': 'require'})


### Paso 1: Ver las primeras filas de cada tabla



In [2]:
tables = ['books', 'authors', 'publishers', 'ratings', 'reviews']
for table in tables:
    query = f"SELECT * FROM {table} LIMIT 5;"
    df = pd.io.sql.read_sql(query, con=engine)
    print(f"Tabla: {table}")
    print(df)


Tabla: books
   book_id  author_id                                              title  \
0        1        546                                       'Salem's Lot   
1        2        465                 1 000 Places to See Before You Die   
2        3        407  13 Little Blue Envelopes (Little Blue Envelope...   
3        4         82  1491: New Revelations of the Americas Before C...   
4        5        125                                               1776   

   num_pages publication_date  publisher_id  
0        594       2005-11-01            93  
1        992       2003-05-22           336  
2        322       2010-12-21           135  
3        541       2006-10-10           309  
4        386       2006-07-04           268  
Tabla: authors
   author_id                          author
0          1                      A.S. Byatt
1          2  Aesop/Laura Harris/Laura Gibbs
2          3                 Agatha Christie
3          4                   Alan Brennert
4          5  

#### **1.2 Paso 1: Ver las primeras filas de cada tabla**

- **Tabla `books`:** La tabla `books` contiene información importante sobre los libros, incluyendo el `book_id`, `author_id`, `title`, `num_pages`, `publication_date`, y `publisher_id`. Esta tabla es esencial para realizar el análisis de los libros, ya que proporciona detalles clave sobre cada libro, como el título y el número de páginas.
  
- **Tabla `authors`:** La tabla `authors` nos da el `author_id` y el nombre del autor. Esto nos permite vincular los libros a sus respectivos autores, facilitando el análisis de la relación entre los libros y sus creadores.

- **Tabla `publishers`:** La tabla `publishers` contiene información sobre las editoriales, incluyendo el `publisher_id` y el nombre de la editorial. Esta información es útil para identificar qué editoriales han publicado los libros.

- **Tabla `ratings`:** La tabla `ratings` nos permite ver las calificaciones de los libros realizadas por los usuarios. Los campos relevantes incluyen el `rating_id`, `book_id`, `username`, y `rating`, lo que facilita el análisis de las calificaciones recibidas por cada libro.

- **Tabla `reviews`:** La tabla `reviews` contiene las reseñas de los libros, con el `review_id`, `book_id`, `username`, y `text` de cada reseña. Esto es importante para evaluar la interacción de los usuarios con los libros a través de sus comentarios escritos.

### Paso 2: Ejercicio 1 - Libros publicados después del 1 de enero de 2000

Ejecutar una consulta SQL para obtener el número de libros publicados después de la fecha indicada:

In [3]:
query_1 = '''
SELECT COUNT(book_id) AS books_after_2000
FROM books
WHERE publication_date > '2000-01-01';
'''

result_1 = pd.io.sql.read_sql(query_1, con=engine)
display(result_1)


,books_after_2000
0,819


#### **1.3 Paso 2: Ejercicio 1 - Libros Publicados Después del 1 de Enero de 2000**

- **Resultado:** Se encontraron **819 libros** que fueron publicados después del 1 de enero de 2000.
  
  **Conclusión:** Un número significativo de libros han sido publicados después del año 2000, lo que refleja un mercado activo y en crecimiento. Esta tendencia puede indicar un aumento en la demanda de nuevos títulos en el mercado, lo que podría ser relevante para un producto o aplicación orientada a amantes de los libros.


### Paso 3: Ejercicio 2 - Número de reseñas de usuarios y calificación promedio por libro

Consulta para analizar las reseñas de los usuarios y las calificaciones promedio:

In [4]:
query_2 = '''
SELECT
    books.book_id,
    books.title,
    COUNT(reviews.review_id) AS num_reviews,
    ROUND(AVG(ratings.rating), 2) AS avg_rating
FROM books
LEFT JOIN reviews ON books.book_id = reviews.book_id
LEFT JOIN ratings ON books.book_id = ratings.book_id
GROUP BY books.book_id, books.title
ORDER BY num_reviews DESC;
'''

result_2 = pd.io.sql.read_sql(query_2, con=engine)
display(result_2)


,book_id,title,num_reviews,avg_rating
0,948,Twilight (Twilight #1),1120,3.66
1,750,The Hobbit or There and Back Again,528,4.13
2,673,The Catcher in the Rye,516,3.83
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,492,4.41
4,299,Harry Potter and the Chamber of Secrets (Harry...,480,4.29
...,...,...,...,...
995,221,Essential Tales and Poems,0,4.00
996,808,The Natural Way to Draw,0,3.00
997,191,Disney's Beauty and the Beast (A Little Golden...,0,4.00
998,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67


  **Conclusión:** Los libros más populares (como "Twilight" y "Harry Potter") tienen un alto número de reseñas, lo que indica una gran interacción de los lectores con estos títulos. Además, los libros con mayor número de reseñas tienden a tener una calificación promedio sólida, lo que refleja una valoración positiva por parte de los usuarios.


### Paso 4: Ejercicio 3 - Editorial con mayor número de libros (más de 50 páginas)

Consulta para encontrar la editorial con más libros de más de 50 páginas:

In [5]:
query_3 = '''
SELECT
    publishers.publisher,
    COUNT(books.book_id) AS num_books
FROM books
JOIN publishers ON books.publisher_id = publishers.publisher_id
WHERE books.num_pages > 50
GROUP BY publishers.publisher
ORDER BY num_books DESC
LIMIT 1;
'''

result_3 = pd.io.sql.read_sql(query_3, con=engine)
display(result_3)


,publisher,num_books
0,Penguin Books,42


#### **1.5 Paso 4: Ejercicio 3 - Editorial con Mayor Número de Libros (Más de 50 Páginas)**

- **Resultado:** La editorial con más libros de más de 50 páginas es **Penguin Books** con **42 libros** publicados.

  **Conclusión:** Penguin Books es una editorial importante en el mercado de libros, con una gran cantidad de títulos que cumplen con el criterio de tener más de 50 páginas. Esto refleja su presencia dominante en el mercado de libros más largos y, posiblemente, más detallados o complejos.


### Ejercicio 4: Autor con la Más Alta Calificación Promedio (Libros con al menos 50 Calificaciones)

Objetivo:

Identificar al autor que tiene la calificación promedio más alta, pero solo para los libros que tienen al menos 50 calificaciones. Este ejercicio es importante para encontrar a los autores cuyos libros han sido más apreciados por los lectores.

In [6]:
query_4 = '''
SELECT
    authors.author,
    ROUND(AVG(ratings.rating), 2) AS avg_rating
FROM books
JOIN authors ON books.author_id = authors.author_id
JOIN ratings ON books.book_id = ratings.book_id
GROUP BY authors.author
HAVING COUNT(ratings.rating) >= 50
ORDER BY avg_rating DESC
LIMIT 1;
'''

result_4 = pd.io.sql.read_sql(query_4, con=engine)
display(result_4)


,author,avg_rating
0,Diana Gabaldon,4.3


In [7]:
query_4_updated = '''
SELECT
    authors.author,
    ROUND(AVG(ratings.rating), 2) AS avg_rating
FROM (
    SELECT
        ratings.book_id
    FROM ratings
    GROUP BY ratings.book_id
    HAVING COUNT(ratings.rating_id) >= 50
) AS filtered_books
JOIN books ON filtered_books.book_id = books.book_id
JOIN authors ON books.author_id = authors.author_id
JOIN ratings ON books.book_id = ratings.book_id
GROUP BY authors.author
ORDER BY avg_rating DESC
LIMIT 1;
'''

result_4_updated = pd.io.sql.read_sql(query_4_updated, con=engine)
display(result_4_updated)


,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.29


#### **1.6 Ejercicio 4: Autor con la Más Alta Calificación Promedio (Libros con al menos 50 Calificaciones)**

- **Resultado:** El autor con la calificación promedio más alta (para libros con al menos 50 calificaciones) es J.K. Rowling/Mary GrandPré, con una calificación promedio de 4.29. Autores de esta categoría representan una oportunidad clave para el desarrollo de productos enfocados en comunidades de fans leales y apasionados, quienes pueden ser receptivos a nuevas iniciativas, como aplicaciones o productos relacionados con su obra.

  **Conclusión:** J.K. Rowling, en colaboración con Mary GrandPré, se destaca como un autor icónico que ha recibido consistentemente altas calificaciones por parte de los lectores. Autores de esta categoría representan una oportunidad clave para el desarrollo de productos enfocados en comunidades de fans leales y apasionados, quienes pueden ser receptivos a nuevas iniciativas, como aplicaciones o productos relacionados con su obra.


### Ejercicio 5: Número Promedio de Reseñas de Texto entre Usuarios que Calificaron Más de 50 Libros

Objetivo: 

Encontrar el número promedio de reseñas de texto que los usuarios han dejado para los libros que calificaron, pero solo para aquellos usuarios que han calificado más de 50 libros. Este ejercicio es importante para identificar los usuarios más activos que interactúan con los libros y dejar reseñas detalladas.

In [8]:
query_5 = '''
SELECT
    ROUND(AVG(CASE WHEN reviews.text IS NOT NULL THEN 1 ELSE 0 END), 2) AS avg_reviews
FROM ratings
JOIN reviews ON ratings.book_id = reviews.book_id
WHERE ratings.username IN (
    SELECT username
    FROM ratings
    GROUP BY username
    HAVING COUNT(ratings.book_id) > 50
);
'''

result_5 = pd.io.sql.read_sql(query_5, con=engine)
display(result_5)


,avg_reviews
0,1.0


In [9]:
query_5_updated = '''
WITH active_users AS (
    SELECT 
        username
    FROM ratings
    GROUP BY username
    HAVING COUNT(book_id) > 50
),
user_reviews AS (
    SELECT 
        ratings.username,
        COUNT(reviews.review_id) AS total_reviews
    FROM ratings
    JOIN reviews ON ratings.book_id = reviews.book_id
    WHERE ratings.username IN (SELECT username FROM active_users)
    GROUP BY ratings.username
)
SELECT 
    ROUND(AVG(total_reviews), 2) AS avg_reviews
FROM user_reviews;
'''

result_5_updated = pd.io.sql.read_sql(query_5_updated, con=engine)
display(result_5_updated)


,avg_reviews
0,211.33


#### **1.7.1 Ejercicio 5: Número Promedio de Reseñas de Texto entre Usuarios que Calificaron Más de 50 Libros**

- **Resultado:** El número promedio de reseñas de texto dejadas por usuarios que calificaron más de 50 libros es de **211.33**.

  **Conclusión:** Los usuarios que califican más de 50 libros muestran un nivel extraordinariamente alto de interacción, dejando en promedio más de 200 reseñas de texto. Este comportamiento refleja un fuerte compromiso con la comunidad y los títulos que califican. Identificar a estos usuarios altamente activos es crucial para diseñar estrategias que fomenten su fidelidad y potencien el contenido generado por los usuarios en la plataforma. Estos usuarios también pueden servir como embajadores clave para atraer y retener a nuevos lectores.


## **Resumen General:**

- **Tendencias en el mercado de libros:** Un número significativo de libros (819 títulos) han sido publicados después del 1 de enero de 2000, lo que refleja un mercado editorial activo y en constante expansión. Editoriales destacadas como Penguin Books lideran con una amplia variedad de publicaciones (42 libros con más de 50 páginas), consolidando su posición en el mercado. 


- **Interacción de los usuarios:** Los libros más populares, como Twilight con 1120 reseñas y una calificación promedio de 3.66, y The Hobbit con 528 reseñas y una calificación promedio de 4.13, muestran que ciertos títulos generan una gran cantidad de interacciones. Esto resalta el compromiso de la comunidad lectora con los títulos más reconocidos.


- **Autores destacados:** Autoras como J.K. Rowling/Mary GrandPré lideran las calificaciones promedio más altas (4.29) para libros con al menos 50 reseñas, demostrando su relevancia y aprecio por parte de los lectores. Este tipo de autores representan oportunidades clave para maximizar la conexión con la audiencia.


- **Comportamiento de los usuarios:** Los usuarios más activos, que calificaron más de 50 libros, dejan en promedio 211.33 reseñas de texto, un indicador de alto compromiso y participación en las plataformas. Este comportamiento puede ser aprovechado para potenciar el contenido generado por los usuarios y fomentar recomendaciones de libros más personalizadas.


- **Conclusión General:** El mercado de libros muestra un dinamismo notable en las publicaciones recientes y un público lector altamente interactivo y comprometido. Autores reconocidos y editoriales prominentes tienen un impacto considerable en las tendencias del mercado. Además, el comportamiento de los usuarios más activos puede ser un recurso valioso para enriquecer las plataformas de reseñas y fortalecer la relación entre lectores y libros.